<a href="https://colab.research.google.com/github/MTajuddin4u/Assigment-04/blob/main/QR_code_encoder_decoder_Python_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pyzbar

In [19]:
import qrcode
from PIL import Image
import cv2
from pyzbar.pyzbar import decode
import numpy as np
import os

class QRCodeProcessor:
    def __init__(self):
        self.qr = qrcode.QRCode(
            version=1,
            error_correction=qrcode.constants.ERROR_CORRECT_L,
            box_size=10,
            border=4,
        )

    def encode_to_qr(self, data, filename="qrcode.png", fill_color="black", back_color="white"):
        """Generate a QR code from text data and save as image"""
        try:
            self.qr.add_data(data)
            self.qr.make(fit=True)

            img = self.qr.make_image(fill_color=fill_color, back_color=back_color)
            img.save(filename)
            print(f"QR code generated and saved as {filename}")
            return True
        except Exception as e:
            print(f"Error generating QR code: {e}")
            return False

    def decode_from_qr(self, image_path):
        """Extract data from a QR code image"""
        try:
            # Read the image
            image = cv2.imread(image_path)

            # Convert to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Detect and decode QR codes
            decoded_objects = decode(gray)

            if not decoded_objects:
                print("No QR code found in the image")
                return None

            results = []
            for obj in decoded_objects:
                results.append({
                    "data": obj.data.decode("utf-8"),
                    "type": obj.type,
                    "rect": obj.rect,
                    "polygon": obj.polygon
                })

            return results
        except Exception as e:
            print(f"Error decoding QR code: {e}")
            return None

    def decode_from_camera(self):
        """Read QR code from webcam in real-time"""
        cap = cv2.VideoCapture(0)
        detected = False

        print("Scanning for QR codes... Press 'q' to quit")

        while True:
            _, frame = cap.read()

            # Convert to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect QR codes
            decoded_objects = decode(gray)

            # Draw bounding boxes around detected QR codes
            for obj in decoded_objects:
                detected = True
                points = obj.polygon
                if len(points) > 4:
                    hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
                    hull = list(map(tuple, np.squeeze(hull)))
                else:
                    hull = points

                # Draw the polygon
                n = len(hull)
                for j in range(n):
                    cv2.line(frame, hull[j], hull[(j+1) % n], (0, 255, 0), 3)

                # Display decoded data
                cv2.putText(frame, obj.data.decode("utf-8"),
                            (hull[0][0], hull[0][1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Display the frame
            cv2.imshow("QR Code Scanner", frame)

            # Exit on 'q' key press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            # Exit if QR code detected
            if detected:
                cv2.waitKey(2000)  # Show result for 2 seconds
                break

        cap.release()
        cv2.destroyAllWindows()

        if detected and decoded_objects:
            return [{"data": obj.data.decode("utf-8"), "type": obj.type}
                   for obj in decoded_objects]
        return None

def main():
    qr_processor = QRCodeProcessor()

    while True:
        print("\nQR Code Encoder/Decoder")
        print("1. Generate QR Code")
        print("2. Decode QR Code from Image")
        print("3. Scan QR Code from Camera")
        print("4. Exit")

        choice = input("Enter your choice (1-4): ")

        if choice == "1":
            data = input("Enter text/data to encode: ")
            filename = input("Enter output filename (default: qrcode.png): ") or "qrcode.png"
            qr_processor.encode_to_qr(data, filename)

        elif choice == "2":
            image_path = input("Enter image path: ")
            if not os.path.exists(image_path):
                print("File not found!")
                continue

            result = qr_processor.decode_from_qr(image_path)
            if result:
                print("\nDecoded Data:")
                for i, item in enumerate(result, 1):
                    print(f"{i}. {item['data']} (Type: {item['type']})")

        elif choice == "3":
            result = qr_processor.decode_from_camera()
            if result:
                print("\nDecoded Data:")
                for i, item in enumerate(result, 1):
                    print(f"{i}. {item['data']} (Type: {item['type']})")

        elif choice == "4":
            print("Exiting...")
            break

        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


QR Code Encoder/Decoder
1. Generate QR Code
2. Decode QR Code from Image
3. Scan QR Code from Camera
4. Exit
Enter your choice (1-4): 1
Enter text/data to encode: play
Enter output filename (default: qrcode.png): ground
QR code generated and saved as ground

QR Code Encoder/Decoder
1. Generate QR Code
2. Decode QR Code from Image
3. Scan QR Code from Camera
4. Exit
Enter your choice (1-4): 4
Exiting...
